In [1]:
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import Convolution1D, MaxPooling1D
from keras.layers import LSTM, SimpleRNN, GRU
from keras.datasets import imdb

Using Theano backend.


In [14]:
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.cross_validation import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import cross_val_score

In [2]:
import numpy as np
import pickle

In [5]:
seed = 7

#Read Data

In [8]:
with open('../Data/train_arrays_1.pickle', 'rb') as handle:
  train_matrix = pickle.load(handle)
with open('../Data/test_arrays_1.pickle', 'rb') as handle:
  test_matrix = pickle.load(handle)

In [18]:
with open('../Data/train_label.pickle', 'rb') as handle:
  y_train = pickle.load(handle)
with open('../Data/test_label.pickle', 'rb') as handle:
  y_test = pickle.load(handle)

##Baseline Model - doc2vec followed Multi Layer Percetrons 

In [31]:
np.random.seed(seed)
model = Sequential()
model.add(Dense(50, input_dim=100, init='normal', activation='relu')) 
model.add(Dense(25, init='normal', activation='relu'))
model.add(Dense(1, init='normal', activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 

In [ ]:
# Fit the model
model.fit(train_matrix, y_train, validation_data=(test_matrix, y_test), nb_epoch=20, batch_size=100,
    verbose=1)
# Final evaluation of the model
scores = model.evaluate(test_matrix, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100)) 

##Word Embedding (with pretrained model)

In [11]:
#from gensim.models.word2vec import Word2Vec
#word_vectors_fname = '../Data/GoogleNews-vectors-negative300.bin.gz'

In [8]:
def word2index(tokens):
    indexes = np.zeros(len(tokens), dtype=int)
    for i, token in enumerate(tokens):
        try:
            indexes[i] = w2v.vocab[token].index
        except KeyError:
            pass
    return indexes

In [9]:
def index2vector(indexes, word2vec_model):
    samples = indexes.shape[0]
    tokens = indexes.shape[1]
    dim = word2vec_model.syn0.shape[1]
    x = np.ndarray((samples, 1, tokens, dim), dtype=object)
    for i in range(samples):
        doc = np.ndarray((tokens, dim))
        for j in range(tokens):
            doc[j] = word2vec_model.syn0[indexes[i][j]]
        x[i] = doc
    return x

In [10]:
def vectorize(boilerplate, nb_tokens, word2vec_model):
    tokenized = [word2index(text_to_word_sequence(t)) for t in boilerplate]
    padded = pad_sequences(tokenized, maxlen=nb_tokens)
    return index2vector(padded, word2vec_model)

In [ ]:
w2v.syn0.shape

In [ ]:
embedding_dim = 300
nb_rows = 1
nb_tokens = 500
nb_feature_maps = 100
ngram_filters = [3, 4, 5, 6, 7, 8]
batch_size = 32
nb_epoch = 2

In [ ]:
X_train = vectorize(X_train, nb_tokens, w2v）

In [ ]:
print('X_train:')
(type(X_train), X_train.shape, X_train.shape[1:], y_train.shape)

#Word embedding(simple one word2vec)

In [22]:
import nltk
from nltk import FreqDist
import pandas as pd
import itertools
import numpy as np

In [2]:
data = pd.read_csv('../Data/allans_forDL.csv', header=None, delimiter="\t",names=['ID','Body','Source','Quality'])

In [3]:
data.head()

,ID,Body,Source,Quality
0,AnsID:5,the r-project http: www.r-project.org (refli...,test,good
1,AnsID:9,"incanter (reflink) is a clojure-based, r-like...",train,good
2,AnsID:12,"see my response to ""datasets for running stati...",train,good
3,AnsID:13,machine learning seems to have its basis in th...,train,good
4,AnsID:14,i second that jay. why is r valuable? here's a...,test,good


In [4]:
train = data[data['Source']=='train']

In [5]:
train.shape

(44598, 4)

In [6]:
sentences = itertools.chain(*[nltk.sent_tokenize(x.decode('utf-8').lower()) for x in data.Body])

In [7]:
tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences]

In [8]:
word_fred = nltk.FreqDist(itertools.chain(*tokenized_sentences))
print "Found %d unique words tokens." % len(word_fred.items())

Found 99562 unique words tokens.


In [9]:
vobcabulary_size = 8000
vocab = word_fred.most_common(vobcabulary_size-1)
index_to_word = [x[0] for x in vocab]
index_to_word.append("UNKNOWN_TOKEN")
word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])

In [10]:
print "The lease frequent word in our vocabulary is '%s' and appeared %d times"% (vocab[-1][0],vocab[-1][1])

The lease frequent word in our vocabulary is 'succinctly' and appeared 37 times


In [11]:
#whole_corpus = ' '.join(data['Body'])
#tokens = nltk.word_tokenize(whole_corpus)
#fdist = nltk.FreqDist(tokens)
#print "Found %d unique words tokens." % len(fdist.items())
#fdist.most_common(8000)[7999]

Found 99637 unique words tokens.


("'success", 37)

In [19]:
train_index = []
for ans in train['Body']:
    tmp = []
    word_ls = nltk.word_tokenize(ans)
    for x in word_ls:
        if word_to_index.has_key(x):
            tmp.append(word_to_index[x])
        else:
            tmp.append(word_to_index["UNKNOWN_TOKEN"])
    train_index.append(tmp)


In [20]:
train_index[0]

[7999,
 11,
 32,
 9,
 5,
 6,
 7999,
 2,
 7999,
 5118,
 11,
 2671,
 490,
 2737,
 9,
 15,
 187,
 885,
 10,
 2382,
 1]

In [36]:
test = data[data['Source']=='test']
test_index = []
for ans in test['Body']:
    tmp = []
    word_ls = nltk.word_tokenize(ans)
    for x in word_ls:
        if word_to_index.has_key(x):
            tmp.append(word_to_index[x])
        else:
            tmp.append(word_to_index["UNKNOWN_TOKEN"])
    test_index.append(tmp)

In [37]:
len(test_index)

29733

##Prepare dataset

In [39]:
import pickle
maxlen = 100
X_train = np.array(train_index)
X_test = np.array(test_index)
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
with open('../Data/X_train.pickle', 'wb') as handle:
    pickle.dump(X_train,handle)
with open('../Data/X_test.pickle', 'wb') as handle:
    pickle.dump(X_test,handle)

# Build a LSTM RNN model

In [33]:
with open('../Data/X_train.pickle', 'rb') as handle:
  X_train = pickle.load(handle)
with open('../Data/X_test.pickle', 'rb') as handle:
  X_test = pickle.load(handle)

In [4]:
with open('../Data/train_label.pickle', 'rb') as handle:
  y_train = pickle.load(handle)
with open('../Data/test_label.pickle', 'rb') as handle:
  y_test = pickle.load(handle)

In [22]:
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

In [18]:
type(X_train)

numpy.ndarray

In [21]:
y_test.shape

(29733,)

In [38]:
max_features = 8001
maxlen = 100

model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))
model.add(LSTM(128))
model.add(Dropout(0.5))
model.add(Dense(10, init='normal', activation='relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [39]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [40]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_3 (Embedding)          (None, 100, 128)      1024128     embedding_input_3[0][0]          
____________________________________________________________________________________________________
lstm_3 (LSTM)                    (None, 128)           131584      embedding_3[0][0]                
____________________________________________________________________________________________________
dropout_3 (Dropout)              (None, 128)           0           lstm_3[0][0]                     
____________________________________________________________________________________________________
dense_4 (Dense)                  (None, 10)            1290        dropout_3[0][0]                  
___________________________________________________________________________________________

In [41]:
model.fit(X_train, y_train, batch_size=100, nb_epoch=4, validation_data=(X_test, y_test),verbose=1)

Train on 44598 samples, validate on 29733 samples
Epoch 1/4
44598/44598 [==============================] - 320s - loss: 0.6829 - acc: 0.5669 - val_loss: 0.7356 - val_acc: 0.4191
Epoch 2/4
44598/44598 [==============================] - 311s - loss: 0.6679 - acc: 0.5951 - val_loss: 0.7714 - val_acc: 0.3923
Epoch 3/4
44598/44598 [==============================] - 303s - loss: 0.6315 - acc: 0.6466 - val_loss: 0.8449 - val_acc: 0.3924
Epoch 4/4
44598/44598 [==============================] - 326s - loss: 0.5863 - acc: 0.6926 - val_loss: 0.9476 - val_acc: 0.3946


In [42]:
score, acc = model.evaluate(X_test, y_test, batch_size=100)
print('Test score:', score)
print('Test accuracy:', acc)

29733/29733 [==============================] - 45s    
('Test score:', 0.94759032315372016)
('Test accuracy:', 0.39464567767115571)


In [32]:
y_train.dtype

dtype('float32')